In [1]:
import xcast as xc 
import xarray as xr 
import cartopy.crs as ccrs 

model = xr.open_dataset('data/MODEL_JJA.nc').prec
obs = xr.open_dataset('data/OBS_JJA.nc').precip.expand_dims({'M':[0]})

In [2]:
model = xc.regrid(model, obs.longitude, obs.latitude)

In [ ]:
model = model.sel(longitude=slice(-120, -70), latitude=slice(20,60))
obs = obs.sel(longitude=slice(-120, -70), latitude=slice(20,60))

drymask = xc.drymask(obs, dry_threshold=30, quantile_threshold=0.3)
obs = obs*drymask 

hindcasts_det, hindcasts_prob = [], []
i=1
for xtrain, ytrain, xtest, ytest in xc.CrossValidator(model, obs, window=3):
    print("window {}".format(i))
    i += 1
    epoelm = xc.EPOELM()
    epoelm.fit(xtrain, ytrain)
    probs =  epoelm.predict_proba(xtest)
    preds = epoelm.predict(xtest)
    hindcasts_det.append(preds.isel(time=1))
    hindcasts_prob.append(probs.isel(time=1))
hindcasts_det = xr.concat(hindcasts_det, 'time')
hindcasts_prob = xr.concat(hindcasts_prob, 'time')


window 1
window 2
window 3
window 4
window 5
window 6
window 7


In [ ]:
ohc = xc.OneHotEncoder() 
ohc.fit(obs)
T = ohc.transform(obs)
clim = xr.ones_like(T) * 0.333

In [ ]:
clim_rps = xc.RankProbabilityScore(clim, T)
pred_rps = xc.RankProbabilityScore(hindcasts_prob, T)
rpss = 1 - pred_rps / clim_rps 

In [ ]:
pl = rpss.plot(subplot_kws={'projection': ccrs.PlateCarree()})
c = pl.axes.coastlines()

In [ ]:
pearson = xc.Pearson(hindcasts_det, obs)

In [ ]:
pl = pearson.plot(subplot_kws={'projection': ccrs.PlateCarree()})
c = pl.axes.coastlines()